In [1]:
import tensorflow as tf


import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

import os

import numpy as np
import pandas as pd
from window_generator import WindowGenerator

from run_automation_utils import Params, init_model


In [2]:
def prep_window_generator(station_id):
    df = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    return WindowGenerator(input_width=24 * 7, label_width=24, shift=24, df=df.copy(), label_columns=["pm10"])


In [3]:
# window_generator_814 = prep_window_generator(814)
# window_generator_530 = prep_window_generator(530)
window_generator_538 = prep_window_generator(538)


In [4]:
from typing import Dict


run_params: Dict[str, Params] = {}
# run_params["station_530_7lstmlayer"] = Params(
#     window_generator_530, lstm_layer_count=7, lstm_sizes=[32, 32, 32, 32, 32, 32, 32]
# )
run_params["station_538_7lstmlayer"] = Params(
    window_generator_538, lstm_layer_count=7, lstm_sizes=[32, 32, 32, 32, 32, 32, 32]
)


In [5]:
import logging

logger = logging.getLogger("run_logger")
fileHandler = logging.FileHandler("run.logs.txt")
logger.addHandler(fileHandler)


In [6]:
for run_id, params in run_params.items():
    logger.error("Start run: " + run_id)
    model = init_model(params)

    loss = tf.keras.losses.MeanSquaredError()
    if params.loss_function == "huber":
        loss = tf.keras.losses.huber
    model.compile(loss=loss, optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    name = params.to_name(100)
    temp_name = name
    append_index = 0
    while os.path.exists("../models/" + temp_name):
        append_index += 1
        temp_name = name + "_" + str(append_index)
    if append_index != 0:
        name += "_" + str(append_index)

    run = neptune.init_run(
        project="data-mining-team2/model-tests",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4Mzg2ZWZmYi05YzRlLTQ3ODYtOWE1NC1mNDM4OTM1ZjNlOTkifQ==",
        custom_run_id=run_id,
        name=name,
        source_files=["./model_tests.ipynb"],
    )

    run["parameters"] = params.to_dict()

    neptune_cbk = NeptuneCallback(run=run, base_namespace="training")

    try:
        for epoch_count_factor in range(1, 5):
            epoch_count = 50 * epoch_count_factor
            history = model.fit(
                params.window_generator.train,
                epochs=50,
                validation_data=params.window_generator.val,
                callbacks=[neptune_cbk],
            )

            eval_metrics = model.evaluate(params.window_generator.test, verbose=0)
            for j, metric in enumerate(eval_metrics):
                run[f"eval/epoch_{epoch_count}/{model.metrics_names[j]}"] = metric

            name = params.to_name(epoch_count)
            temp_name = name
            append_index = 0
            while os.path.exists("../models/" + temp_name):
                append_index += 1
                temp_name = name + "_" + str(append_index)
            if append_index != 0:
                name += "_" + str(append_index)

            model.save("../models/" + name, include_optimizer=False)

            run["model_names/" + str(epoch_count) + "epochs"] = name
    except Exception as e:
        logger.error(e)

        pass
    run.stop()
    logger.error("Finished run: " + run_id)


e:\FHBielefeld\Master\DataMining\feinstaubprojekt-polen\.env\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-5


e:\FHBielefeld\Master\DataMining\feinstaubprojekt-polen\.env\lib\site-packages\neptune\common\warnings.py:62: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'window_generator.WindowGenerator'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  warnings.warn(


Epoch 1/50
682/682 [==============================] - 21s 15ms/step - loss: 206.1052 - mean_absolute_error: 10.0551 - val_loss: 130.7104 - val_mean_absolute_error: 7.9739
Epoch 2/50
682/682 [==============================] - 9s 13ms/step - loss: 131.2832 - mean_absolute_error: 8.0987 - val_loss: 131.8184 - val_mean_absolute_error: 8.2992
Epoch 3/50
682/682 [==============================] - 9s 13ms/step - loss: 128.9623 - mean_absolute_error: 8.0326 - val_loss: 137.1435 - val_mean_absolute_error: 8.6315
Epoch 4/50
682/682 [==============================] - 9s 13ms/step - loss: 128.4685 - mean_absolute_error: 8.0198 - val_loss: 130.8272 - val_mean_absolute_error: 7.9302
Epoch 5/50
682/682 [==============================] - 9s 13ms/step - loss: 128.2346 - mean_absolute_error: 8.0340 - val_loss: 130.7465 - val_mean_absolute_error: 8.2284
Epoch 6/50
682/682 [==============================] - 9s 13ms/step - loss: 122.5429 - mean_absolute_error: 7.8471 - val_loss: 123.6963 - val_mean_absolut

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_50_epochs_7lstm-layer_32,32,32,32,32,32,32_dropout-0-4_1\assets


...

Epoch 1/50
682/682 [==============================] - 11s 17ms/step - loss: 71.0642 - mean_absolute_error: 5.9656 - val_loss: 85.2957 - val_mean_absolute_error: 6.5437
Epoch 2/50
682/682 [==============================] - 12s 18ms/step - loss: 72.0435 - mean_absolute_error: 5.9945 - val_loss: 88.9891 - val_mean_absolute_error: 6.6301
Epoch 3/50
682/682 [==============================] - 12s 18ms/step - loss: 70.6855 - mean_absolute_error: 5.9450 - val_loss: 84.3916 - val_mean_absolute_error: 6.4784
Epoch 4/50
682/682 [==============================] - 12s 18ms/step - loss: 71.2701 - mean_absolute_error: 5.9635 - val_loss: 87.0118 - val_mean_absolute_error: 6.5676
Epoch 5/50
682/682 [==============================] - 12s 17ms/step - loss: 72.2862 - mean_absolute_error: 6.0098 - val_loss: 86.1155 - val_mean_absolute_error: 6.5680
Epoch 6/50
682/682 [==============================] - 12s 17ms/step - loss: 71.7687 - mean_absolute_error: 5.9686 - val_loss: 86.1562 - val_mean_absolute_error:

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_7lstm-layer_32,32,32,32,32,32,32_dropout-0-4_1\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_7lstm-layer_32,32,32,32,32,32,32_dropout-0-4_1\assets


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-5/metadata


ERROR:run_logger:Finished run: station_538_7lstmlayer
